## Download YOLOv8:

In [ ]:
!git clone -b final https://github.com/borsym/FootballInsight.git
!cp -r FootballInsight/src/complete/ /content/
!rm -rf FootballInsight
!mv /content/complete/* /content/

In [ ]:
!git clone --recurse-submodules https://github.com/mikel-brostrom/yolov8_tracking.git
!mkdir /content/yolov8
!cp -r /content/yolov8_tracking/yolov8/* /content/yolov8
!rm -rf /content/yolov8_tracking/

## Download Demo Video:

In [ ]:
## 10 sec video
!gdown https://drive.google.com/uc?id=1t0EctgtEbiW5WEv6gZkTMw2TjQlDjkry

## Download Requirements:

In [ ]:
!pip install -qr requirements.txt
!pip install thop
!pip install sentry_sdk
!pip install imageio[ffmpeg]
!pip install imageio[pyav]

## Download Model Files:

In [ ]:
## Pre-Trained YOLOv8x.pt model for object detection.
!gdown https://drive.google.com/uc?id=1CJjmDetBADOd6DUIPgSgZUghqEGF3EcS 
## YOLOv8x-pose-p6.pt model for pose estimation.
!gdown https://drive.google.com/uc?id=1AlZ61lNy_C9-cdU_9E7YQEaf_v82yh3N 
## OSNET model.
!gdown https://drive.google.com/uc?id=1EZLd0x9_ldqDpPFgVGtxRExa9SnxJyXx

## Run Inference:

In [ ]:
%cd /content/
!python track.py --yolo-weights yolov8x_custom.pt --reid-weights osnet_x0_25_msmt17.pt --source quality.mp4 --save-vid --imgsz 1920 --conf 0.2 --classes 0 1 2 3

## Display Video:

In [ ]:
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

def display_video(video):
    fig = plt.figure(figsize=(8,8))

    mov = []
    for i in range(len(video)): 
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])

    anime = animation.ArtistAnimation(fig, mov, interval=50, repeat_delay=1000)

    plt.close()
    return anime

In [ ]:
video = imageio.mimread(f'./runs/track/exp/quality.mp4', memtest=False)
HTML(display_video(video).to_html5_video())